In [1]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score

In [ ]:
client = OpenAI(api_key = 'sk-proj-')

In [3]:
df = pd.read_csv('Heart_failure_clinical_records_dataset.csv')


target = df['DEATH_EVENT']
features = df.drop(columns=['DEATH_EVENT'])


In [4]:
features_sample = features.head(300)
target_sample = target.head(300)

In [5]:
def llm_predict(row):
    prompt = f"""You are given the clinical data of a patient suffering from heart failure. 
Your task is to predict whether the patient will experience a death event (binary classification: 0 = no, 1 = yes).

Analyze the patient's full clinical profile carefully, considering all these factors:

- Age: Higher age increases mortality risk.
- Anaemia: 1 = Anemia present, 0 = No anemia. Anemia increases risk.
- Creatinine Phosphokinase: Very high levels may indicate heart or muscle injury.
- Diabetes: 1 = Diabetic, 0 = Non-diabetic. Diabetes worsens heart outcomes.
- Ejection Fraction: Lower percentages mean poorer heart function.
- High Blood Pressure: 1 = Hypertension present, 0 = Normal blood pressure.
- Platelets: Abnormal platelet counts may suggest additional risks.
- Serum Creatinine: Elevated levels indicate kidney dysfunction.
- Serum Sodium: Low sodium suggests worse heart failure condition.
- Sex: 1 = Male, 0 = Female. Slight differences in mortality risk.
- Smoking: 1 = Smoker, 0 = Non-smoker. Smoking worsens outcomes.
- Time: Follow-up period in days. Longer survival indicates better prognosis if no early death.

Follow this step-by-step reasoning process:

1. Identify and assess the patient's risk factors based on the above features.
2. Evaluate whether the combination of factors suggests a high or low risk of mortality.
3. Make a final prediction: 0 (no death event) or 1 (death event).

Examples:

---

Patient Data:
- Age: 80
- Anaemia: 1
- Creatinine Phosphokinase: 250
- Diabetes: 1
- Ejection Fraction: 20
- High Blood Pressure: 1
- Platelets: 150000
- Serum Creatinine: 3.0
- Serum Sodium: 130
- Sex: 1
- Smoking: 1
- Time: 10

Reasoning:
- Very high age (80 years).
- Anemia is present.
- Moderate CPK elevation (250 mcg/L).
- Diabetes is present.
- Extremely low ejection fraction (20%).
- Hypertension is present.
- Platelet count normal.
- High serum creatinine suggests kidney dysfunction.
- Low sodium (130) indicates hyponatremia.
- Male and smoker status increase risk.
- Very short follow-up time (10 days) hints acute situation.

Prediction: 1 (death event)

---

Patient Data:
- Age: 45
- Anaemia: 0
- Creatinine Phosphokinase: 120
- Diabetes: 0
- Ejection Fraction: 55
- High Blood Pressure: 0
- Platelets: 270000
- Serum Creatinine: 0.9
- Serum Sodium: 140
- Sex: 0
- Smoking: 0
- Time: 250

Reasoning:
- Relatively young age (45 years).
- No anemia.
- Normal CPK level.
- No diabetes.
- Normal ejection fraction (55%), indicating good heart function.
- No hypertension.
- Normal platelet count.
- Normal kidney function (serum creatinine 0.9).
- Normal serum sodium (140).
- Non-smoker, female.
- Very long follow-up time (250 days) without adverse events.

Prediction: 0 (no death event)

---

Now, here is the patient's data you need to predict:

- Age: {row['age']}
- Anaemia: {row['anaemia']}
- Creatinine Phosphokinase: {row['creatinine_phosphokinase']}
- Diabetes: {row['diabetes']}
- Ejection Fraction: {row['ejection_fraction']}
- High Blood Pressure: {row['high_blood_pressure']}
- Platelets: {row['platelets']}
- Serum Creatinine: {row['serum_creatinine']}
- Serum Sodium: {row['serum_sodium']}
- Sex: {row['sex']}
- Smoking: {row['smoking']}
- Time: {row['time']}

Please provide your step-by-step reasoning and final prediction.
"""

    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4"
        messages=[
            {"role": "system", "content": "You are a medical assistant specialized in analyzing heart failure patient data to predict death events."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    output = response.choices[0].message.content.strip()
    return int('1' in output)


In [6]:
preds = features_sample.apply(llm_predict, axis=1)

In [7]:
acc = accuracy_score(target_sample, preds)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.32
